In [1]:
import os
import pandas as pd
os.chdir('..')
from src.models.entmoot_functions import *
from src.models.train_animation_predictor import *

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### 1. Load and preprocess training data for optimizer fitting

#### Load data

In [3]:
X_train = pd.read_csv('data/path_selector/path_selector_train.csv')

In [4]:
X_train.head()

,filename,animation_id,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9,stroke_width,opacity,stroke_opacity,fill_r,fill_g,fill_b,stroke_r,stroke_g,stroke_b,svg_fill_r,diff_fill_r,svg_fill_g,diff_fill_g,svg_fill_b,diff_fill_b,svg_stroke_r,diff_stroke_r,svg_stroke_g,diff_stroke_g,svg_stroke_b,diff_stroke_b,rel_width,rel_height,rel_x_position,rel_y_position,nr_paths_svg
0,logo_0,1,13.535693,6.971131,-0.009867,-0.915823,-2.984741,5.383109,0.495111,-2.146576,4.868704,-0.944599,0.0,1.0,1.0,197,0,123,0,0,0,238.6,-41.6,162.6,-162.6,127.4,-4.4,0.0,0.0,0.0,0.0,0.0,0.0,0.400180,0.472019,0.700090,0.564323,10
1,logo_0,0,-1.000982,4.641413,1.455743,-0.487705,-1.037431,6.984421,-0.485484,-3.933893,6.030271,-1.227060,0.0,1.0,1.0,250,187,0,0,0,0,238.6,11.4,162.6,24.4,127.4,-127.4,0.0,0.0,0.0,0.0,0.0,0.0,0.500000,0.671553,0.250000,0.335776,10
2,logo_0,2,10.590673,0.533370,8.743198,0.241546,-2.132272,0.620374,1.687153,6.592582,3.034471,-1.564105,0.0,1.0,1.0,236,116,4,0,0,0,238.6,-2.6,162.6,-46.6,127.4,-123.4,0.0,0.0,0.0,0.0,0.0,0.0,0.787194,0.328324,0.499991,0.835838,10
3,logo_0,3,4.222565,-0.735711,5.308626,-0.090910,-4.875907,2.410124,0.314957,-1.771255,3.811236,-0.731849,0.0,1.0,1.0,197,0,123,0,0,0,238.6,-41.6,162.6,-162.6,127.4,-4.4,0.0,0.0,0.0,0.0,0.0,0.0,0.400206,0.472019,0.299897,0.564323,10
4,logo_0,4,1.776277,8.866785,-2.770646,-0.955766,-4.725605,5.412009,0.597616,2.331442,2.209505,-1.606922,0.0,1.0,1.0,236,116,4,0,0,0,238.6,-2.6,162.6,-46.6,127.4,-123.4,0.0,0.0,0.0,0.0,0.0,0.0,0.586900,0.315475,0.500106,0.170381,10


In [5]:
X_train = retrieve_m1_predictions(X_train)
X_train = retrieve_animation_midpoints(X_train, drop=True)

2021-05-09 23:46:43::INFO::Number of animated paths: 2141/8921
2021-05-09 23:46:43::INFO::Non-animated paths won't be used for training
2021-05-09 23:46:43::INFO::Start extraction midpoint of animated paths as feature
data/initial_svgs/logo_253.svg, animation ID 24: svg2path fails and path bbox cannot be computed. list index out of range
data/initial_svgs/logo_253.svg: svg2path fails and bbox of multiple paths cannot be computed. list index out of range
data/initial_svgs/logo_253.svg, animation_id 24, animated_animation_ids [24, 25, 27, 26]: rel_x_position not defined and set to 0.5. float division by zero
data/initial_svgs/logo_253.svg, animation_id 24, animated_animation_ids [24, 25, 27, 26]: rel_y_position not defined and set to 0.5. float division by zero
data/initial_svgs/logo_253.svg, animation ID 25: svg2path fails and path bbox cannot be computed. list index out of range
data/initial_svgs/logo_253.svg: svg2path fails and bbox of multiple paths cannot be computed. list index out

In [6]:
scaler = pickle.load(open(config.scaler_path, 'rb'))
X_train[config.sm_features] = scaler.transform(X_train[config.sm_features])

In [7]:
X_train.head()

,filename,animation_id,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9,stroke_width,opacity,stroke_opacity,fill_r,fill_g,fill_b,stroke_r,stroke_g,stroke_b,svg_fill_r,diff_fill_r,svg_fill_g,diff_fill_g,svg_fill_b,diff_fill_b,svg_stroke_r,diff_stroke_r,svg_stroke_g,diff_stroke_g,svg_stroke_b,diff_stroke_b,rel_width,rel_height,rel_x_position,rel_y_position,nr_paths_svg,animated,animated_animation_ids,rel_position_to_animations,rel_x_position_to_animations,rel_y_position_to_animations
0,logo_0,1,1.104144,1.051156,0.162952,-0.291088,-0.560505,1.310510,0.057168,-0.996996,2.843173,-0.794722,0.0,1.0,1.0,0.404193,-1.478231,0.103801,0,0,0,1.575065,-0.819546,0.74144,-2.213786,0.276719,-0.111728,0.0,0.0,0.0,0.0,0.0,0.0,0.623730,0.512336,1.243948,0.707014,-0.759252,1,"[1, 0, 2, 3, 4, 5]","(0.7000899895197011, 0.5643229197828795)",0.776532,0.298184
1,logo_0,0,-0.781468,0.644867,0.448649,-0.206212,0.135806,1.920836,-0.567073,-1.785888,3.519717,-1.025105,0.0,1.0,1.0,0.919725,0.656063,-1.240189,0,0,0,1.575065,-0.262998,0.74144,0.107133,0.276719,-1.670432,0.0,0.0,0.0,0.0,0.0,0.0,1.058472,1.255520,-0.685434,-0.445612,-0.759252,1,"[1, 0, 2, 3, 4, 5]","(0.25, 0.33577636718263004)",-1.038886,-0.720511
2,logo_0,2,0.722133,-0.071553,1.869222,-0.061634,-0.255682,-0.504764,0.816016,2.860326,1.774841,-1.300009,0.0,1.0,1.0,0.783547,-0.154284,-1.196482,0,0,0,1.575065,-0.410010,0.74144,-0.774072,0.276719,-1.619742,0.0,0.0,0.0,0.0,0.0,0.0,2.309276,-0.022874,0.386193,2.076345,-0.759252,1,"[1, 0, 2, 3, 4, 5]","(0.4999911632674701, 0.8358382216582674)",-0.030558,1.508403
3,logo_0,3,-0.103900,-0.292873,1.199707,-0.127545,-1.236740,0.177383,-0.057516,-0.831335,2.227261,-0.621197,0.0,1.0,1.0,0.404193,-1.478231,0.103801,0,0,0,1.575065,-0.819546,0.74144,-2.213786,0.276719,-0.111728,0.0,0.0,0.0,0.0,0.0,0.0,0.623844,0.512336,-0.471543,0.707014,-0.759252,1,"[1, 0, 2, 3, 4, 5]","(0.2998969019667724, 0.5643229197828794)",-0.837629,0.298184
4,logo_0,4,-0.421218,1.381748,-0.375218,-0.299007,-1.182995,1.321525,0.122423,0.979528,1.294347,-1.334932,0.0,1.0,1.0,0.783547,-0.154284,-1.196482,0,0,0,1.575065,-0.410010,0.74144,-0.774072,0.276719,-1.619742,0.0,0.0,0.0,0.0,0.0,0.0,1.436946,-0.070728,0.386685,-1.279748,-0.759252,1,"[1, 0, 2, 3, 4, 5]","(0.5001060407903573, 0.1703812636898346)",-0.030094,-1.457723


#### Merge path animation vectors and labels

In [6]:
with open('data/surrogate_model/animation_label.pkl', 'rb') as f:
    animations = pickle.load(f)

In [9]:
animations.head()

,file,animation_id,order_id,path_prob,begin_value,model_output,animated_animation_ids,animated_order_ids,backend_mapping,logo_id,animation_number,alias,animation_file,time,label
0,logo_316_animation_0,4,1,0.2,1.00,"[0, 0, 0, 1, 0, 0, -1.0, -1.0, -1.0, -1.0, 0.8...","[4, 24, 23]","[1, 6, 7]","[0, 1, 0, 0, 0, 0, 1, 1]",316,0,Jakob,animation/logo_316_animation_0.svg,"Timestamp(seconds=1617783014, nanoseconds=3260...",Good
1,logo_316_animation_0,24,6,0.6,1.25,"[0, 0, 0, 0, 0, 1, -1.0, -1.0, -1.0, -1.0, -1....","[4, 24, 23]","[1, 6, 7]","[0, 1, 0, 0, 0, 0, 1, 1]",316,0,Jakob,animation/logo_316_animation_0.svg,"Timestamp(seconds=1617783014, nanoseconds=3260...",Good
2,logo_316_animation_0,23,7,0.2,1.50,"[0, 0, 0, 0, 1, 0, -1.0, -1.0, -1.0, -1.0, -1....","[4, 24, 23]","[1, 6, 7]","[0, 1, 0, 0, 0, 0, 1, 1]",316,0,Jakob,animation/logo_316_animation_0.svg,"Timestamp(seconds=1617783014, nanoseconds=3260...",Good
3,logo_316_animation_1,4,1,0.2,1.00,"[1, 0, 0, 0, 0, 0, 0.13436424411240122, 0.8474...","[4, 24, 23]","[1, 6, 7]","[0, 1, 0, 0, 0, 0, 1, 1]",316,1,Jonathan,animation/logo_316_animation_1.svg,"Timestamp(seconds=1617813571, nanoseconds=9700...",Very Bad
4,logo_316_animation_1,24,6,0.6,1.25,"[0, 0, 1, 0, 0, 0, -1.0, -1.0, -1.0, 0.7637746...","[4, 24, 23]","[1, 6, 7]","[0, 1, 0, 0, 0, 0, 1, 1]",316,1,Jonathan,animation/logo_316_animation_1.svg,"Timestamp(seconds=1617813571, nanoseconds=9700...",Okay


In [10]:
# get correct filenames to match
animations['filename'] = animations['file'].apply(lambda row: row.split("_animation")[0])

In [11]:
# drop unnecessary columns
animations.drop(['file','order_id','path_prob','begin_value','animated_animation_ids','animated_order_ids','backend_mapping','logo_id','animation_number','alias','animation_file', 'time'], axis=1, inplace=True)

# map label names to label indexes and delete entries without rating
mapping = {'Very Bad': 0, 'Bad': 1, 'Okay': 2, 'Good': 3, 'Very Good': 4}
animations.replace({'label': mapping}, inplace=True)
animations = animations[animations['label'] != 'no_rating']
animations.reset_index(drop=True, inplace=True)

In [12]:
animations.head()

,animation_id,model_output,label,filename
0,4,"[0, 0, 0, 1, 0, 0, -1.0, -1.0, -1.0, -1.0, 0.8...",3,logo_316
1,24,"[0, 0, 0, 0, 0, 1, -1.0, -1.0, -1.0, -1.0, -1....",3,logo_316
2,23,"[0, 0, 0, 0, 1, 0, -1.0, -1.0, -1.0, -1.0, -1....",3,logo_316
3,4,"[1, 0, 0, 0, 0, 0, 0.13436424411240122, 0.8474...",0,logo_316
4,24,"[0, 0, 1, 0, 0, 0, -1.0, -1.0, -1.0, 0.7637746...",2,logo_316


In [13]:
initial_data = animations.merge(X_train, how='left', on=['filename', 'animation_id'])

In [14]:
print(f"Before: Train: {initial_data.shape}.")
initial_data.dropna(inplace=True)
print(f"After: Train: {initial_data.shape}.")

Before: Train: (22462, 45).
After: Train: (9671, 45).


In [15]:
# Drop unnecessary columns
initial_data.drop(['stroke_width','opacity','stroke_opacity','stroke_r','stroke_g','stroke_b','svg_stroke_r','diff_stroke_r','svg_stroke_g','diff_stroke_g','svg_stroke_b','diff_stroke_b'], axis=1, inplace=True)

In [16]:
initial_data.head()

,animation_id,model_output,label,filename,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9,fill_r,fill_g,fill_b,svg_fill_r,diff_fill_r,svg_fill_g,diff_fill_g,svg_fill_b,diff_fill_b,rel_width,rel_height,rel_x_position,rel_y_position,nr_paths_svg,animated,animated_animation_ids,rel_position_to_animations,rel_x_position_to_animations,rel_y_position_to_animations
0,4,"[0, 0, 0, 1, 0, 0, -1.0, -1.0, -1.0, -1.0, 0.8...",3,logo_316,-2.246351,0.441433,-0.368473,0.219164,0.835400,-1.043020,2.176880,-0.352190,-0.042259,-0.684932,-0.928409,-0.793431,-0.584585,-0.717208,-0.430400,-0.74687,-0.252072,-0.672931,-0.113523,-0.880700,-0.931991,-1.612048,-1.926608,0.02123,1.0,"[20, 19, 23, 21, 24, 22, 4]","(0.03383765572036357, 0.04211967340054534)",-1.910767,-2.029420
1,24,"[0, 0, 0, 0, 0, 1, -1.0, -1.0, -1.0, -1.0, -1....",3,logo_316,0.337741,0.634562,1.545425,-1.272139,-0.308202,1.646020,0.430931,-0.196219,-0.346564,-0.013042,0.112382,0.427796,0.584577,-0.717208,0.693198,-0.74687,1.075940,-0.672931,1.242423,0.605501,0.105866,0.392709,0.782491,0.02123,1.0,"[20, 19, 23, 21, 24, 22, 4]","(0.5015112311156477, 0.5792887719245869)",-0.024426,0.364891
2,23,"[0, 0, 0, 0, 1, 0, -1.0, -1.0, -1.0, -1.0, -1....",3,logo_316,0.477755,0.497751,1.737961,-1.249104,-0.199626,1.685413,0.507097,-0.201009,-0.258875,0.019934,0.112382,0.427796,0.584577,-0.717208,0.693198,-0.74687,1.075940,-0.672931,1.242423,0.605501,0.105925,0.392709,1.078284,0.02123,1.0,"[20, 19, 23, 21, 24, 22, 4]","(0.5015112311156477, 0.6379395431595825)",-0.024426,0.626314
3,4,"[1, 0, 0, 0, 0, 0, 0.13436424411240122, 0.8474...",0,logo_316,-2.246351,0.441433,-0.368473,0.219164,0.835400,-1.043020,2.176880,-0.352190,-0.042259,-0.684932,-0.928409,-0.793431,-0.584585,-0.717208,-0.430400,-0.74687,-0.252072,-0.672931,-0.113523,-0.880700,-0.931991,-1.612048,-1.926608,0.02123,1.0,"[20, 19, 23, 21, 24, 22, 4]","(0.03383765572036357, 0.04211967340054534)",-1.910767,-2.029420
4,24,"[0, 0, 1, 0, 0, 0, -1.0, -1.0, -1.0, 0.7637746...",2,logo_316,0.337741,0.634562,1.545425,-1.272139,-0.308202,1.646020,0.430931,-0.196219,-0.346564,-0.013042,0.112382,0.427796,0.584577,-0.717208,0.693198,-0.74687,1.075940,-0.672931,1.242423,0.605501,0.105866,0.392709,0.782491,0.02123,1.0,"[20, 19, 23, 21, 24, 22, 4]","(0.5015112311156477, 0.5792887719245869)",-0.024426,0.364891


#### Transform animation vector into multiple dataframe columns and change column ordering

In [17]:
initial_data[[f'an_vec_{i}' for i in range(12)]] = pd.DataFrame(initial_data['model_output'].tolist(), index=initial_data.index)

initial_data.drop(['model_output','animation_id','filename'], inplace=True, axis=1)

In [18]:
col_order = [f'an_vec_{i}' for i in range(12)] + [f'emb_{i}' for i in range(10)] + ['_'.join(['fill', ch]) for ch in ['r','g','b']] + ['_'.join(['svg_fill', ch]) for ch in ['r','g','b']] + ['_'.join(['diff_fill', ch]) for ch in ['r','g','b']] + ['rel_height','rel_width','rel_x_position','rel_y_position','rel_x_position_to_animations','rel_y_position_to_animations','nr_paths_svg','label']
initial_data = initial_data[col_order]

#### Encode labels into 4-binary labels

In [19]:
initial_data[[f'rating_{i}' for i in range(4)]] = pd.DataFrame(encode_classes(np.array(initial_data['label'])), index=initial_data.index)

initial_data.drop(['label'], axis=1, inplace=True)

initial_data.reset_index(drop=True, inplace=True)

In [20]:
initial_data.head()

,an_vec_0,an_vec_1,an_vec_2,an_vec_3,an_vec_4,an_vec_5,an_vec_6,an_vec_7,an_vec_8,an_vec_9,an_vec_10,an_vec_11,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9,fill_r,fill_g,fill_b,svg_fill_r,svg_fill_g,svg_fill_b,diff_fill_r,diff_fill_g,diff_fill_b,rel_height,rel_width,rel_x_position,rel_y_position,rel_x_position_to_animations,rel_y_position_to_animations,nr_paths_svg,rating_0,rating_1,rating_2,rating_3
0,0,0,0,1,0,0,-1.000000,-1.000000,-1.0,-1.000000,0.844422,0.757954,-2.246351,0.441433,-0.368473,0.219164,0.835400,-1.043020,2.176880,-0.352190,-0.042259,-0.684932,-0.928409,-0.793431,-0.584585,-0.717208,-0.74687,-0.672931,-0.430400,-0.252072,-0.113523,-0.931991,-0.880700,-1.612048,-1.926608,-1.910767,-2.029420,0.02123,1,1,1,0
1,0,0,0,0,0,1,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,0.337741,0.634562,1.545425,-1.272139,-0.308202,1.646020,0.430931,-0.196219,-0.346564,-0.013042,0.112382,0.427796,0.584577,-0.717208,-0.74687,-0.672931,0.693198,1.075940,1.242423,0.105866,0.605501,0.392709,0.782491,-0.024426,0.364891,0.02123,1,1,1,0
2,0,0,0,0,1,0,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,0.477755,0.497751,1.737961,-1.249104,-0.199626,1.685413,0.507097,-0.201009,-0.258875,0.019934,0.112382,0.427796,0.584577,-0.717208,-0.74687,-0.672931,0.693198,1.075940,1.242423,0.105925,0.605501,0.392709,1.078284,-0.024426,0.626314,0.02123,1,1,1,0
3,1,0,0,0,0,0,0.134364,0.847434,-1.0,-1.000000,-1.000000,-1.000000,-2.246351,0.441433,-0.368473,0.219164,0.835400,-1.043020,2.176880,-0.352190,-0.042259,-0.684932,-0.928409,-0.793431,-0.584585,-0.717208,-0.74687,-0.672931,-0.430400,-0.252072,-0.113523,-0.931991,-0.880700,-1.612048,-1.926608,-1.910767,-2.029420,0.02123,0,0,0,0
4,0,0,1,0,0,0,-1.000000,-1.000000,-1.0,0.763775,-1.000000,-1.000000,0.337741,0.634562,1.545425,-1.272139,-0.308202,1.646020,0.430931,-0.196219,-0.346564,-0.013042,0.112382,0.427796,0.584577,-0.717208,-0.74687,-0.672931,0.693198,1.075940,1.242423,0.105866,0.605501,0.392709,0.782491,-0.024426,0.364891,0.02123,1,1,0,0


In [21]:
initial_data.to_csv('data/entmoot_optimization/entmoot_initial_data_full.csv', index=False)

### 2. Train Bayesian optimizer

In [15]:
# Load subset of initial data (100 training examples)
initial_data = pd.read_csv('data/entmoot_optimization/entmoot_initial_data_reduced.csv')

In [16]:
initial_data.head()

,an_vec_0,an_vec_1,an_vec_2,an_vec_3,an_vec_4,an_vec_5,an_vec_6,an_vec_7,an_vec_8,an_vec_9,an_vec_10,an_vec_11,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9,fill_r,fill_g,fill_b,svg_fill_r,svg_fill_g,svg_fill_b,diff_fill_r,diff_fill_g,diff_fill_b,rel_height,rel_width,rel_x_position,rel_y_position,rel_x_position_to_animations,rel_y_position_to_animations,nr_paths_svg,rating_0,rating_1,rating_2,rating_3
0,0,0,0,0,0,1,-1.000000,-1.000000,-1.0,-1.000000,-1.0,-1.0,0.337741,0.634562,1.545425,-1.272139,-0.308202,1.646020,0.430931,-0.196219,-0.346564,-0.013042,0.112382,0.427796,0.584577,-0.717208,-0.74687,-0.672931,0.693198,1.075940,1.242423,0.105866,0.605501,0.392709,0.782491,-0.024426,0.364891,0.02123,1,1,1,0
1,0,0,0,0,1,0,-1.000000,-1.000000,-1.0,-1.000000,-1.0,-1.0,0.477755,0.497751,1.737961,-1.249104,-0.199626,1.685413,0.507097,-0.201009,-0.258875,0.019934,0.112382,0.427796,0.584577,-0.717208,-0.74687,-0.672931,0.693198,1.075940,1.242423,0.105925,0.605501,0.392709,1.078284,-0.024426,0.626314,0.02123,1,1,1,0
2,1,0,0,0,0,0,0.134364,0.847434,-1.0,-1.000000,-1.0,-1.0,-2.246351,0.441433,-0.368473,0.219164,0.835400,-1.043020,2.176880,-0.352190,-0.042259,-0.684932,-0.928409,-0.793431,-0.584585,-0.717208,-0.74687,-0.672931,-0.430400,-0.252072,-0.113523,-0.931991,-0.880700,-1.612048,-1.926608,-1.910767,-2.029420,0.02123,0,0,0,0
3,0,0,1,0,0,0,-1.000000,-1.000000,-1.0,0.544229,-1.0,-1.0,0.337741,0.634562,1.545425,-1.272139,-0.308202,1.646020,0.430931,-0.196219,-0.346564,-0.013042,0.112382,0.427796,0.584577,-0.717208,-0.74687,-0.672931,0.693198,1.075940,1.242423,0.105866,0.605501,0.392709,0.782491,-0.024426,0.364891,0.02123,1,0,0,0
4,0,0,1,0,0,0,-1.000000,-1.000000,-1.0,0.369955,-1.0,-1.0,0.477755,0.497751,1.737961,-1.249104,-0.199626,1.685413,0.507097,-0.201009,-0.258875,0.019934,0.112382,0.427796,0.584577,-0.717208,-0.74687,-0.672931,0.693198,1.075940,1.242423,0.105925,0.605501,0.392709,1.078284,-0.024426,0.626314,0.02123,1,0,0,0


In [17]:
X_train = initial_data.iloc[:, :-4]

# use 0-padding instead of padding with -1 
X_train.replace(to_replace=-1, value=0, inplace=True)

y_train = initial_data.iloc[:, -4:]

y_train = pd.Series(decode_classes(y_train.to_numpy()).flatten()) * -1

X_train = X_train.values.tolist()
y_train = y_train.values.tolist()

In [18]:
# Load surrogate model for function evaluations
func = SurrogateModelFNN()

In [19]:
# Fit optimizer to given initial data

optimizer = None
start_time = time.time()
optimizer = entmoot_fit(dimensions=func.get_bounds(), x0=X_train, y0=y_train, random_state=73, base_estimator="RF", std_estimator="L1DDP")
print("--- %s seconds ---" % (time.time() - start_time))

# Save fitted optimizer
with open(f'models/entmoot_optimizer_final.pkl', 'wb') as output:
    pickle.dump(optimizer, output, pickle.HIGHEST_PROTOCOL)

------------------------------------------------------------
Fitted a model to observed evaluations of the objective.
--- 3.922826051712036 seconds ---


In [20]:
# Show example of animation prediction

# Load optimizer
with open("models/entmoot_optimizer_final.pkl", "rb") as f:
    optimizer = pickle.load(f)

# Predict animations for new path
path_vector = [9.0731e-02,  9.1726e-01, -2.2292e-01,
         1.0261e+00, -1.0310e+00,  3.8478e-01,  4.4583e-01,  3.7610e+00,
         9.0707e-02, -1.5281e+00, -1.1364e+00, -1.0705e+00, -8.6565e-01,
        -7.8171e-01, -9.8579e-01, -8.6640e-01, -6.1583e-01, -3.6518e-01,
        -2.7900e-01, -7.6306e-01,  2.0780e+00,  4.8552e-01, -1.5957e-01,
        -2.7360e-03, -1.3878e+00,  9.0921e-02]

start_time = time.time()
opt_x, opt_y,_ = entmoot_predict(optimizer, func, path_vector, n_calls=1)
print("--- %s seconds ---" % (time.time() - start_time))

print(f'Predicted animation vector: {opt_x}')
print('-'*60)
print(f'Predicted animation score: {-opt_y}')

--- 1.4402360916137695 seconds ---
Predicted animation vector: [0, 0, 0, 0, 1, 0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.090731, 0.91726, -0.22292, 1.0261, -1.031, 0.38478, 0.44583, 3.761, 0.090707, -1.5281, -1.1364, -1.0705, -0.86565, -0.78171, -0.98579, -0.8664, -0.61583, -0.36518, -0.279, -0.76306, 2.078, 0.48552, -0.15957, -0.002736, -1.3878, 0.090921]
------------------------------------------------------------
Predicted animation score: 2
